In [ ]:
!pip install mastml

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os                        # OS stands for Operating System and provides ways for python to interact with files or directories
from collections import Counter  # Collections is a package for handling data
from pprint import pprint

import pandas as pd              # Pandas is a data analysis library which we'll primarily use to handle our dataset
import numpy as np               # Numpy is a package for scientific computing. We'll use it for some of it's math functions
import matplotlib                # Matplotlib is the plotting package that we'll use throughout the lab
import matplotlib.pyplot as plt
import seaborn as sns            # Seaborn is a Python data visualization library based on matplotlib
import sklearn                   # Scikit-learn is a machine learning package, providing the backbone for the work we'll perform
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate,GridSearchCV,ParameterGrid
from sklearn.model_selection import KFold,RepeatedKFold
from mastml.mastml import Mastml
# MastML and other imports
from mastml.mastml import Mastml
from mastml.models import SklearnModel
from mastml.preprocessing import SklearnPreprocessor
from mastml.feature_selectors import SklearnFeatureSelector, EnsembleModelFeatureSelector, ShapFeatureSelector
from mastml.feature_generators import OneHotGroupGenerator
from mastml.data_splitters import SklearnDataSplitter
import numpy as np
from copy import copy
import mastml
import os
try:
    data_path = os.path.join(mastml.__path__._path[0], 'data')
except:
    data_path = os.path.join(mastml.__path__[0], 'data')

scikit-lego is an optional dependency, enabling use of the LowessRegression model. If you want to use this model, do "pip install scikit-lego"
linear-tree is an optional dependency, enabling use of Linear tree, forest, and boosting models. If you want to use this model, do "pip install linear-tree"
gplearn is an optional dependency, enabling the use of genetic programming SymbolicRegressor model. If you want to use this model, do "pip install gplearn"
CBFV is an optional dependency. To install CBFV, do pip install cbfv
DeepChem is an optional dependency used to generate molecular descriptors from RDKit. To install Deepchem, do pip install deepchem


ERROR:duecredit:Failed to import duecredit due to No module named 'duecredit'


In [ ]:
SAVEPATH = 'drive/MyDrive/Food Pantry Research/v3_RF_results'

mastml_instance = Mastml(savepath=SAVEPATH)
savepath = mastml_instance.get_savepath

In [ ]:
from pathlib import Path
file_path = Path('/content/drive/MyDrive/Food Pantry Research')
path = file_path
df = pd.read_csv(os.path.join(path,"skunkworks_data_v3.csv"))
df

,Date,Amount,Type,Fund,Campaign,Appeal,Zip,Account Number,Median Household Income,Population
0,9/25/2024,0.00,Donation,Unrestricted,In-Kind,Individual Multi Channel,53716.0,338,NaN,NaN
1,9/25/2024,300.00,Donation,Unrestricted,Support The River,Individual Online,53704.0,2574,50831.0,44208.0
2,9/25/2024,25.81,Recurring Donation Payment,Unrestricted,Support The River,Individual Online,53590.0,2692,70127.0,37430.0
3,9/24/2024,0.00,Donation,Unrestricted,In-Kind,Business Multi Channel,53716.0,20749,NaN,NaN
4,9/24/2024,100.00,Donation,Unrestricted,Support The River,Individual Multi Channel,53703.0,310,29561.0,28255.0
...,...,...,...,...,...,...,...,...,...,...
9801,1/1/2023,25.00,Recurring Donation Payment,Unrestricted,Support The River,Individual Online,53704.0,8,50831.0,44208.0
9802,1/1/2023,108.15,Donation,Unrestricted,Year-End 2022,Year-End Appeal Mailing 2022,53590.0,244,70127.0,37430.0
9803,1/1/2023,103.00,Donation,Unrestricted,Support The River,Individual Online,53718.0,84,71127.0,12342.0
9804,1/1/2023,103.00,Donation,Unrestricted,Support The River,Individual Online,53711.0,4540,70554.0,45589.0


In [ ]:
df.drop(['Population'], axis=1, inplace=True)
df.drop(['Account Number'], axis=1, inplace=True)
df.drop(['Median Household Income'], axis=1, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Year'] = df['Date'].dt.year
df = df.drop(columns=['Date'])  # Drop original Date column
df

,Amount,Type,Fund,Campaign,Appeal,Zip,Month,Day,Year
0,0.00,Donation,Unrestricted,In-Kind,Individual Multi Channel,53716.0,9,25,2024
1,300.00,Donation,Unrestricted,Support The River,Individual Online,53704.0,9,25,2024
2,25.81,Recurring Donation Payment,Unrestricted,Support The River,Individual Online,53590.0,9,25,2024
3,0.00,Donation,Unrestricted,In-Kind,Business Multi Channel,53716.0,9,24,2024
4,100.00,Donation,Unrestricted,Support The River,Individual Multi Channel,53703.0,9,24,2024
...,...,...,...,...,...,...,...,...,...
9801,25.00,Recurring Donation Payment,Unrestricted,Support The River,Individual Online,53704.0,1,1,2023
9802,108.15,Donation,Unrestricted,Year-End 2022,Year-End Appeal Mailing 2022,53590.0,1,1,2023
9803,103.00,Donation,Unrestricted,Support The River,Individual Online,53718.0,1,1,2023
9804,103.00,Donation,Unrestricted,Support The River,Individual Online,53711.0,1,1,2023


In [ ]:
# One-hot encode categorical features
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['Type', 'Fund', 'Campaign', 'Appeal']

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df[categorical_features])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_features))

In [ ]:
one_hot_df

,Type_Donation,Type_Pledge Payment,Type_Recurring Donation Payment,Type_Recurring Donation Schedule,Type_Soft Credit,Fund_Cap Camp Operational Res,Fund_Cap Camp Unrestricted,Fund_Munch,Fund_Programs/Operations,Fund_Restricted Misc,...,Appeal_Individual Event,Appeal_Individual Mailing,Appeal_Individual Multi Channel,Appeal_Individual Online,Appeal_Individual Payroll,Appeal_Service Org Event,Appeal_Service Org Mailing,Appeal_Service Org Multi Channel,Appeal_Service Org Online,Appeal_Year-End Appeal Mailing 2022
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9801,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9802,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9804,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
featurized_df = pd.concat([df.drop(columns=categorical_features), one_hot_df], axis=1)
featurized_df

,Amount,Zip,Month,Day,Year,Type_Donation,Type_Pledge Payment,Type_Recurring Donation Payment,Type_Recurring Donation Schedule,Type_Soft Credit,...,Appeal_Individual Event,Appeal_Individual Mailing,Appeal_Individual Multi Channel,Appeal_Individual Online,Appeal_Individual Payroll,Appeal_Service Org Event,Appeal_Service Org Mailing,Appeal_Service Org Multi Channel,Appeal_Service Org Online,Appeal_Year-End Appeal Mailing 2022
0,0.00,53716.0,9,25,2024,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,300.00,53704.0,9,25,2024,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25.81,53590.0,9,25,2024,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,53716.0,9,24,2024,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100.00,53703.0,9,24,2024,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9801,25.00,53704.0,1,1,2023,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9802,108.15,53590.0,1,1,2023,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9803,103.00,53718.0,1,1,2023,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9804,103.00,53711.0,1,1,2023,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Separate features and target variable
X = featurized_df.drop(columns=['Amount'])  # Features
y = featurized_df['Amount']  # Target variable

In [ ]:
X

,Zip,Month,Day,Year,Type_Donation,Type_Pledge Payment,Type_Recurring Donation Payment,Type_Recurring Donation Schedule,Type_Soft Credit,Fund_Cap Camp Operational Res,...,Appeal_Individual Event,Appeal_Individual Mailing,Appeal_Individual Multi Channel,Appeal_Individual Online,Appeal_Individual Payroll,Appeal_Service Org Event,Appeal_Service Org Mailing,Appeal_Service Org Multi Channel,Appeal_Service Org Online,Appeal_Year-End Appeal Mailing 2022
0,53716.0,9,25,2024,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,53704.0,9,25,2024,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,53590.0,9,25,2024,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53716.0,9,24,2024,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,53703.0,9,24,2024,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9801,53704.0,1,1,2023,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9802,53590.0,1,1,2023,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9803,53718.0,1,1,2023,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9804,53711.0,1,1,2023,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y

,Amount
0,0.00
1,300.00
2,25.81
3,0.00
4,100.00
...,...
9801,25.00
9802,108.15
9803,103.00
9804,103.00


In [ ]:
model_xgb = SklearnModel(model='XGBoostRegressor', n_estimators=150)
model_rf = SklearnModel(model='RandomForestRegressor', n_estimators=150)
models = [model_rf, model_xgb]


preprocessor = SklearnPreprocessor(preprocessor='StandardScaler', as_frame=True)

baseline_tests = ["test_mean", "test_permuted"]
# Run the evaluation for both models
metrics = ['r2_score', 'mean_absolute_error', 'root_mean_squared_error', 'rmse_over_stdev']
# Set up 5-fold cross-validation
splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=1, n_splits=5)
splitter.evaluate(X=X,
                  y=y,
                  models=models,
                  preprocessor=preprocessor,
                  metrics=metrics,
                  plots=['Scatter', 'Histogram'],
                  savepath=savepath,
                  baseline_test=baseline_tests,
                  verbosity=1)

Running splits: 100%|██████████| 5/5 [18:47<00:00, 225.48s/it]


Running splits: 100%|██████████| 5/5 [00:19<00:00,  3.80s/it]
